In [1]:
%load_ext autoreload
%autoreload 2

from keys import *
from pathlib import Path
from urllib.parse import urlparse

import pinecone
import requests
from langchain import LLMChain, PromptTemplate
from langchain.chains import (ConversationalRetrievalChain,
                              SimpleSequentialChain)
from langchain.document_loaders import (PagedPDFSplitter, TextLoader,
                                        UnstructuredHTMLLoader,
                                        UnstructuredMarkdownLoader,
                                        UnstructuredPDFLoader)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline, OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from keys import HUGGINGFACE_HEADERS
from slackbot import chain

from pprint import PrettyPrinter
pp = PrettyPrinter()

# https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html?highlight=pinecone

got OPENAI_API_KEY
got PINECONE_API_KEY
got PINECONE_ENV
got HUGGINGFACE_API_KEY
got SLACK_TOKEN
got SLACK_SIGNING_SECRET


/home/piotrm/anaconda3/envs/demo3/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
class TestModel():
    def __init__(self):
        # llm = OpenAI()
       
        self.llm_model_id = "gpt2"
        # This model is pretty bad but using it for tests because it is free and
        # relatively small.

        # model_id = "decapoda-research/llama-7b-hf"
        # model_id = "decapoda-research/llama-13b-hf"

        self.model = AutoModelForCausalLM.from_pretrained(
            self.llm_model_id,
            device_map='auto',
            torch_dtype=torch.float16,
            local_files_only=True)

        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_id,
                                                       local_files_only=True)

        self.pipe = pipeline("text-generation",
                             model=self.model,
                             tokenizer=self.tokenizer,
                             max_new_tokens=16,
                             device_map="auto",
                             early_stopping=True)

        self.llm = HuggingFacePipeline(pipeline=self.pipe)

        template = """Q: {question} A:"""
        self.prompt = PromptTemplate(template=template, input_variables=["question"])
        self.llm_chain = LLMChain(prompt=self.prompt, llm=self.llm, verbose=True)

t = TestModel()

In [3]:
from tru_chain import TruChain

In [4]:
tc = TruChain(t.llm_chain)
tc("hello there")

/home/piotrm/anaconda3/envs/demo3/lib/python3.10/site-packages/transformers/generation/utils.py:1253: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt after formatting:
Q: hello there A:


{'question': 'hello there',
 'text': ' so what are i doing now? Are you there, and if so where is'}

In [61]:
import json
from types import NoneType
from typing import Dict, Sequence
import jsonpickle
import pydantic


class MaybeJSONDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.decoded = dict()

    def _build(self, struct):
        print(f"building {struct}")
        if isinstance(struct, Dict) and "_pydantic" in struct:
            metadata = struct['_pydantic']
            print(f"metadata={metadata}")

            data = {k: self._build(v) for k, v in struct.items() if k != "_pydantic"}

            print(data)

            mod = __import__(metadata['module'])
            cls = getattr(mod, metadata['class'])
            return cls.parse_obj(data)

        elif isinstance(struct, Dict):
            data = {k: self._build(v) for k, v in struct.items()}
            return data

        return struct

    def decode(self, s):
        struct = super().decode(s)

        return self._build(struct)


class MaybeJSONEncoder(json.JSONEncoder):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.encoded = dict()

    # @staticmethod
    def default(self, obj):
        if isinstance(obj, (str, int, float, bool, NoneType)):
            return obj
        
        obj_id = id(obj)
        # print(obj_id)

        if obj_id in self.encoded:
            return self.encoded[obj_id]#{'_ref': obj_id}

        self.encoded[obj_id] = {'_ref': obj_id}

        metadata = {
                'class': obj.__class__.__name__,
                'module': obj.__class__.__module__,
                'id': id(obj)
            }

        src = dict()

        print(f"defaulting {type(obj)}")

        if isinstance(obj, pydantic.BaseModel):
            try:
                src = obj.dict(models_as_dict=False)
            except BaseException as e:
                metadata['error'] = str(e)
                
            src['_pydantic'] = metadata
            self.encoded[obj_id] = src

            return src

        elif isinstance(obj, dict):
            return {k: self.default(v) for k, v in obj.items()}
        
        elif isinstance(obj, list):
            return [self.default(v) for v in obj]

        else:
            return {
                '_nonserial': metadata
            }
    
    # @staticmethod
    def encode(self, obj):
        print(f"encoding {obj}")
        obj = self.default(obj)
        return super().encode(obj)

    """
        obj_id = id(obj)
        print(obj_id)

        if obj_id in self.encoded:
            return super().encode({'_ref': obj_id})
        
        # temporary reference
        self.encoded[obj_id] = {'_ref': obj_id}

        if isinstance(obj, Dict):
            ret = {}
            for k, v in obj.items():
                ret[k] = self.encode(v)

        elif isinstance(obj, Sequence):
            ret = []
            for v in obj:
                ret.append(self.encode(v))
        else:
            print(type(obj))
            ret = self.default(obj)
            # ret = super().encode(obj)

        self.encoded[obj_id] = ret

        return super().encode(ret)
    """

import langchain

class TestModel(pydantic.BaseModel):
    
    chain: langchain.chains.base.Chain

    class Config:
        encoder = MaybeJSONEncoder()
        decoder = MaybeJSONDecoder()

        def json_dumps(o, default):
            return TestModel.Config.encoder.encode(o)
        
        def json_loads(s):
            return TestModel.Config.decoder.decode(s)
        
        #json_encoders = {
        #    TestModel: encoder.encode
        #}
        #json_dumps = lambda o, default: Config.encoder.encode(o)

LLMChain.Config = TestModel.Config
tm = TestModel(chain=t.llm_chain)

# json.dumps(tc.model, default=default)
"""
@jsonpickle.handlers.register(pydantic.BaseModel, base=True)
class FooHandler(jsonpickle.handlers.BaseHandler):
    def flatten(self, obj, data):
        print("pydantic", obj, data)
        h = jsonpickle.handlers.get(dict)#(self.context)
        print(h)
        h = h(self.context)
        h.flatten(obj.dict(), data)
        #if isinstance(obj, pydantic.BaseModel):
        #    # h = jsonpickle.handlers.BaseHandler(self.context)
        #    
        #    print(h)
        #    h.flatten(obj, data)
        #else:
        #    pass


@jsonpickle.handlers.register(object, base=True)
class FooHandler(jsonpickle.handlers.BaseHandler):
    def flatten(self, obj, data):
        print("default", obj, data)
        pass    
"""
        
# jsonpickle.handlers.register(object, )

#temp = jsonpickle.dumps(tc.chain)

# temp = jsonpickle.encode(tc.model)

# enc = MaybeJSONEncoder(check_circular=False)
# dec = MaybeJSONDecoder()

# dump = json.dumps(enc.encode(tc.chain))

# dec.decode(dump)
# print(dump)

'\n@jsonpickle.handlers.register(pydantic.BaseModel, base=True)\nclass FooHandler(jsonpickle.handlers.BaseHandler):\n    def flatten(self, obj, data):\n        print("pydantic", obj, data)\n        h = jsonpickle.handlers.get(dict)#(self.context)\n        print(h)\n        h = h(self.context)\n        h.flatten(obj.dict(), data)\n        #if isinstance(obj, pydantic.BaseModel):\n        #    # h = jsonpickle.handlers.BaseHandler(self.context)\n        #    \n        #    print(h)\n        #    h.flatten(obj, data)\n        #else:\n        #    pass\n\n\n@jsonpickle.handlers.register(object, base=True)\nclass FooHandler(jsonpickle.handlers.BaseHandler):\n    def flatten(self, obj, data):\n        print("default", obj, data)\n        pass    \n'

In [69]:
t.llm_chain.save("temp.json")
llm_chain2 = langchain.chains.loading.load_chain_from_config(t.llm_chain.dict())

In [70]:
print(llm_chain2)

memory=None callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa48a49f490> verbose=True prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Q: {question} A:', template_format='f-string', validate_template=True) llm=HuggingFacePipeline(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa48a49f490>, pipeline=None, model_id='gpt2', model_kwargs=None) output_key='text'


In [62]:
print(type(tm))
encoded = tm.json(models_as_dict=False)
print(encoded)
TestModel.parse_raw(encoded)

<class '__main__.TestModel'>
encoding {'chain': LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa48a49f490>, verbose=True, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Q: {question} A:', template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa48a49f490>, pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fa3a4cafeb0>, model_id='gpt2', model_kwargs=None), output_key='text')}
defaulting <class 'dict'>
defaulting <class 'langchain.chains.llm.LLMChain'>
{"chain": {"memory": null, "verbose": true, "prompt": {"input_variables": ["question"], "output_parser": null, "partial_variables": {}, "template": "Q: {question} A:", "template_format": "f-string", "validate_template": true, "_type": "prompt"}, "llm": {"model_id": "g

Traceback (most recent call last):
  File "pydantic/main.py", line 539, in pydantic.main.BaseModel.parse_raw
  File "pydantic/parse.py", line 37, in pydantic.parse.load_str_bytes
  File "/tmp/ipykernel_2260677/2695286400.py", line 139, in json_loads
    return TestModel.Config.decoder.decode(s)
  File "/tmp/ipykernel_2260677/2695286400.py", line 36, in decode
    return self._build(struct)
  File "/tmp/ipykernel_2260677/2695286400.py", line 28, in _build
    data = {k: self._build(v) for k, v in struct.items()}
  File "/tmp/ipykernel_2260677/2695286400.py", line 28, in <dictcomp>
    data = {k: self._build(v) for k, v in struct.items()}
  File "/tmp/ipykernel_2260677/2695286400.py", line 25, in _build
    return cls.parse_obj(data)
  File "pydantic/main.py", line 526, in pydantic.main.BaseModel.parse_obj
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 3 validation errors for LLMChain
prompt
  Can't instantiate abstract c

In [ ]:
len(temp)

In [ ]:
tc.model_dict

In [ ]:
tc.records

In [ ]:
from tru_chain import Selection

tc._select(select=[
    Selection(param=("chain", "prompt", "template")),
    Selection(param=("chain", "llm", "model_id")),
    Selection(record=("input", "inputs", "question"))
])

In [ ]:
tc.model

In [ ]:
template = """Q: {question} A:"""
prompt = PromptTemplate(template=template,
                        input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=t.llm)

template_2 = """Reverse this sentence: {sentence}."""
prompt_2 = PromptTemplate(template=template_2,
                            input_variables=["sentence"])
llm_chain_2 = LLMChain(prompt=prompt_2, llm=t.llm)

seq_chain = SimpleSequentialChain(chains=[llm_chain, llm_chain_2],
                                    input_key="question",
                                    output_key="answer")

tc = TruChain(seq_chain)
tc.model

In [ ]:
tc("hello there")

In [ ]:
tc.records

In [ ]:
tc._get_obj_at_address(address=("chains", 0, ))

In [ ]:
tc.model

# Notes

1. Langchain does not have support for classification models: https://python.langchain.com/en/latest/modules/models.html

    - Will have to figure out out-of-band retrieval and execution of feedback models that are not LLM's.

2. Can add steps to chain to capture text at various points in a chain: https://python.langchain.com/en/latest/reference/modules/chains.html#langchain.chains.SequentialChain .


# Links

- https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/llama#transformers.LlamaForCausalLM

- https://huggingface.co/docs/transformers/main_classes/text_generation


# Pinecone



In [ ]:
from slackbot import chain
import langchain
import dill

In [ ]:
from langchain.chains import (ConversationalRetrievalChain,
                              SimpleSequentialChain)

verb = False

template = """Q: {question} A:"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=t.llm, verbose=verb)

template_2 = """Reverse this sentence: {sentence}."""
prompt_2 = PromptTemplate(template=template_2, input_variables=["sentence"])
llm_chain_2 = LLMChain(prompt=prompt_2, llm=t.llm, verbose=verb)

# print(llm_chain.run(question="What is the average air speed velocity of a laden swallow?"))

print(llm_chain_2.run(sentence="How are you doing?"))

seq_chain = SimpleSequentialChain(chains=[llm_chain, llm_chain_2], input_key="question", output_key="answer")
seq_chain.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2 = TruChain(seq_chain)

In [ ]:
seq_chain.run(question="What is the average air speed velocity of a laden swallow? again")
tru_chain_2.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
for r in tru_chain_2.records:
    print(pp.pformat(r))

In [ ]:
tru_chain_2.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2.model

In [ ]:
import inspect
for frame_info in inspect.stack():
    frame = frame_info.frame
    print(frame_info.function)
    # print(frame.f_code)
    print(frame.f_locals.keys())